# The Hartree Fock method

<div class="col-md-10">
<p> In this tutorial we use a the Python scripting language and an interactive Jupyter notebook to run a Hartree-Fock calculation using the Python-based quantum chemistry code Psi4. </p>


<p> You can install this setup on your own Windows/Mac computer following the instructions in the lecture handout.</p>
</div>
<div class="col-md-2">
<img src="http://www.psicode.org/psi4manual/master/_static/psi4square.png" alt="Psi4 code" style="width: 100px;"/>
</div>

You will see lot's of Python statements that import packages or access functions, such as the following:


Here we load important packages, such as a visualization programm and the Psi4 software package and then we do a little arithmetics.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import psi4
from ase.build import molecule
from ase.atoms import Atoms
from ase.visualize import view
from ase.visualize.plot import plot_atoms

def geom_ase_to_psi4(atoms, charge=0, multiplicity=1, symmetry='c1' ):
    """
    takes an ase atoms object and build a Psi4 geometry string
    """
    species = atoms.get_chemical_symbols()
    pos = atoms.get_positions()
    string = "unit angstrom \n"
    string += "symmetry c1 \n"
    string += "{0} {1} \n".format(charge, multiplicity) 
    for i,(spec,p) in enumerate(zip(species, pos)):
        string += "{0} {1:14.8f} {2:14.8f} {3:14.8f}\n".format(spec, p[0], p[1], p[2])
        
    return string

print(1+1)

2


## Part 1: Theory and Method

Hartree-Fock Self-Consistent Field Theory (HF-SCF) with restricted orbitals and closed-shell systems (RHF) solves the so-called Rothaan-Hall equation, which is a generalized eigenvalue matrix equation:

\begin{equation}
\mathbf{FC}=\mathbf{\epsilon S C}
\end{equation}


## Part 2: A simple Hartree-Fock calculation

### Step 1: Setting up the molecular coordinates
___________

This is the first step of every calculation. We need to define the molecular coordinates. 
The `ase` package contains a lot of handy tools to define molecules with preset geometries.

In the following, we use the functino `molecules` from `ase.build` to build a water molecule.

In [35]:

#h2o, nh3, BDA, C60, 
atoms = molecule('H2')
print(atoms.positions)

[[ 0.        0.        0.368583]
 [ 0.        0.       -0.368583]]


We could have also done this manually, by building an `Atoms` object and setting Cartesian coordinates by hand:

In [20]:

atoms = Atoms('OH2')
atoms.set_positions([
    [0.,     0.,  0.119262], #H
    [0.,  0.763, -0.477047], #H
    [0., -0.763, -0.477047], #O
])
print(atoms.positions)

[[ 0.        0.        0.119262]
 [ 0.        0.763    -0.477047]
 [ 0.       -0.763    -0.477047]]


Let's visualize the molecule on the screen using `ase.visualize.view`
There are two possible viewers. 'x3d' is the standard one. 'ngl' or nglview has to be installed separately (see handout) but has a lot more features. Try to replace 

`view(atoms, viewer='x3d')` with `view(atoms, viewer='ngl')`

In [37]:

view(atoms,viewer='x3d') #x3d or ngl

### Step 2: Performing the calculation
___________
To calculate the HF energy, we use Psi4. In the following,

* we set the RAM memory, which we allow Psi4 to use, 
* we pass the geometry from our `ase.atoms` object to `psi4.geometry`
* we initiate the HF (also called scf) calculation using a split-valence 6-31G* Pople basis set
* we perform the HF energy calculation using `psi4.energy`
* we extract and print important output data

In [16]:
psi4.core.clean()
psi4.core.set_output_file('output.dat', False)
psi4.set_memory('1000 MB')

##set geometry
#transform ase geometries to Psi4 input string
geom_input = geom_ase_to_psi4(atoms, charge=0, multiplicity=1)
print(geom_input)
#initiate Psi4 molecule object
nh3 = psi4.geometry(geom_input)

# Set computation options
psi4.set_options({'basis': '3-21G',
                  'reference': 'rhf',
                  'scf_type':'direct',
                  'e_convergence': 1e-8})

E, wvf = psi4.energy('scf',return_wfn=True)
print('Total HF energy is {0} Hartree \n '.format(E))

#Let's look at orbital coefficients, MO energies and occupations
coeffs = wvf.Ca().to_array()
epsilons = wvf.epsilon_a().to_array()
occs = wvf.occupation_a().to_array()

for i,(o,e) in enumerate(zip(occs, epsilons)):
    print ('Molecular Orbital {0}, occupation: {1:4.2f}, energy: {2:8.3f}'.format(i,o,e))

#Write wave functions to molden file format, can be visualized in Avogadro
psi4.molden(wvf, 'C60.molden')


unit angstrom 
symmetry c1 
0 1 
C     2.21019530     0.58666310     2.66695040
C     3.10763930     0.15770080     1.63002860
C     1.32844300    -0.31589390     3.23632320
C     3.09087090    -1.15850050     1.20142400
C     3.18792450    -1.45745990    -0.19970050
C     3.22146230     1.22309660     0.67394400
C     3.31612100     0.93515860    -0.67651510
C     3.29849810    -0.43011420    -1.12041380
C    -0.44808420     1.35914840     3.20810200
C     0.46720560     2.29498300     2.61752640
C    -0.02565750     0.07642190     3.50862590
C     1.77279170     1.91765840     2.35296910
C     2.39546230     2.30956890     1.11895390
C    -0.26101950     3.08209350     1.66231170
C     0.34077260     3.45923880     0.47459680
C     1.69511710     3.06924460     0.19766230
C    -2.12583940    -0.84588530     2.67009630
C    -2.56209900     0.48552020     2.35317150
C    -0.87815210    -1.04619850     3.23673020
C    -1.74150960     1.56799630     2.61973330
C    -1.62624680     2.6357

Molecular Orbital 500, occupation: 0.00, energy:    1.935
Molecular Orbital 501, occupation: 0.00, energy:    1.937
Molecular Orbital 502, occupation: 0.00, energy:    1.937
Molecular Orbital 503, occupation: 0.00, energy:    1.938
Molecular Orbital 504, occupation: 0.00, energy:    1.938
Molecular Orbital 505, occupation: 0.00, energy:    2.197
Molecular Orbital 506, occupation: 0.00, energy:    2.197
Molecular Orbital 507, occupation: 0.00, energy:    2.198
Molecular Orbital 508, occupation: 0.00, energy:    2.198
Molecular Orbital 509, occupation: 0.00, energy:    2.199
Molecular Orbital 510, occupation: 0.00, energy:    2.228
Molecular Orbital 511, occupation: 0.00, energy:    2.230
Molecular Orbital 512, occupation: 0.00, energy:    2.230
Molecular Orbital 513, occupation: 0.00, energy:    2.237
Molecular Orbital 514, occupation: 0.00, energy:    2.238
Molecular Orbital 515, occupation: 0.00, energy:    2.239
Molecular Orbital 516, occupation: 0.00, energy:    2.475
Molecular Orbi

## H2 binding energy curve

Total HF energy: 0.4 -0.9506786580943305 Hartree
Total HF energy: 0.6 -1.1319534605884076 Hartree
Total HF energy: 0.8 -1.150027881209585 Hartree
Total HF energy: 0.9 -1.1406024516864692 Hartree
Total HF energy: 1.0 -1.1267783519580556 Hartree
Total HF energy: 1.1 -1.1112698792995999 Hartree
Total HF energy: 1.4 -1.0667752729709794 Hartree
Total HF energy: 1.8 -1.0261357171893148 Hartree
Total HF energy: 2.5 -1.0008131478562312 Hartree
Total HF energy: 3.5 -0.996689871523255 Hartree
Total HF energy: 4.5 -0.9964731737747944 Hartree
Total HF energy: 6 -0.9964658254839056 Hartree
Total HF energy: 10 -0.9964658214581413 Hartree


RuntimeError: 
Fatal Error: Iterations did not converge.
Error occurred in file: /scratch/psilocaluser/conda-builds/psi4_1495014881710/work/psi4/src/psi4/libparallel/process.cc on line: 194
The most recent 5 function calls were:

psi::PsiException::PsiException(std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >, char const*, int)
psi::die_if_not_converged()
psi::scf::HF::finalize_E()
psi::scf::HF::compute_energy()
